# JS-Working
This notebook is to test JS functions and work on graph algos

## Requirements
Install Deno Jupyter Kernel: https://docs.deno.com/runtime/manual/tools/jupyter/

In [1]:
console.log("Hello")

Hello


### fileUtils

In [2]:
async function fileHandler(file){
    const text = await readFileAsync(file)
    return text
};

// Helper function to read async
function readFileAsync(file) {
    return new Promise((resolve, reject) => {
        let reader = new FileReader();

        reader.onload = () => {
            resolve(reader.result);
        };

        reader.onerror = reject;

        reader.readAsText(file);
    })
}

### graph.svelte functions

In [3]:
import N3 from 'n3'
const { namedNode, defaultGraph } = N3.DataFactory

In [4]:
class GraphStore {
    graph;
    parser = new N3.Parser()

    constructor(){
        this.graph = new N3.Store()
    }

    // Will make this a $derived action on text change in future, 
    // but for now (and speed) will just manually trigger on: file upload | UI Button
    async parse(fileAsString, namedgraph="https://graph.com/unnamed#"){
        // TODO: namedgraph should be typed / error checked.
        const namedGraphNode = namedNode(namedgraph)

        await this.parser.parse(fileAsString, (error, quad) => {
            if (quad) {
                this.graph.addQuad(quad.subject, quad.predicate, quad.object, namedGraphNode)
            } else if (error) {
                console.log("Error parsing: ", error)
            } else {
                console.log("Parsing complete.") // TODO: Remove this in prod and do something smarter
            }
        });
    }

    // Helper methods for processing SHACL result graph
    getResultNodes() {
        /// Returns all top level validation results
        /// Nested result detail needs to be expanded for each top level result
        return this.graph.getObjects(null, namedNode("http://www.w3.org/ns/shacl#result"), null)
    }

    // Couple of DEBUG helper methods
    _node(uri) { return namedNode(uri) }
    get _default() { return defaultGraph() }
    get _namedGraphs() { return Object.keys(this.graph?._graphs) }
}

## Working

In [5]:
// Load file into memory as text
// FILE UTILS are not needed as Deno can read using its built in API :)
const fileData = await Deno.readTextFile("/Users/wcrd/Desktop/report_test_graph_2.ttl")

In [6]:
let graphStore = new GraphStore();
await graphStore.parse(fileData)

Parsing complete.


In [7]:
graphStore.getResultNodes()

[
  BlankNode { id: "_:n3-1" },
  BlankNode { id: "_:n3-3" },
  BlankNode { id: "_:n3-5" },
  BlankNode { id: "_:n3-12" },
  BlankNode { id: "_:n3-13" },
  BlankNode { id: "_:n3-15" },
  BlankNode { id: "_:n3-16" },
  BlankNode { id: "_:n3-18" },
  BlankNode { id: "_:n3-20" },
  BlankNode { id: "_:n3-22" },
  BlankNode { id: "_:n3-23" },
  BlankNode { id: "_:n3-25" },
  BlankNode { id: "_:n3-26" },
  BlankNode { id: "_:n3-27" },
  BlankNode { id: "_:n3-29" },
  BlankNode { id: "_:n3-31" },
  BlankNode { id: "_:n3-33" },
  BlankNode { id: "_:n3-35" },
  BlankNode { id: "_:n3-37" },
  BlankNode { id: "_:n3-39" },
  BlankNode { id: "_:n3-41" },
  BlankNode { id: "_:n3-43" },
  BlankNode { id: "_:n3-45" },
  BlankNode { id: "_:n3-47" },
  BlankNode { id: "_:n3-49" },
  BlankNode { id: "_:n3-51" },
  BlankNode { id: "_:n3-53" },
  BlankNode { id: "_:n3-55" },
  BlankNode { id: "_:n3-56" },
  BlankNode { id: "_:n3-57" },
  BlankNode { id: "_:n3-59" },
  BlankNode { id: "_:n3-60" },
  BlankNo

## Tree Generator
Now that we have the graph data available in here, lets write an ag-grid tree generator

EG from ag-grid.com

```javascript
const rowData = [
    { orgHierarchy: ['Erica'], jobTitle: "CEO", employmentType: "Permanent" },
    { orgHierarchy: ['Erica', 'Malcolm'], jobTitle: "VP", employmentType: "Permanent" }
    ...
]
```

Need to provide path as array


In [8]:
/* 
Plan
--------
for each top level result:
    enrich
    recursively decend detail until no more
        for each detail, enrich
*/

function enrichNode(store, node){
    // get all pred-objs
    const quads = store.getQuads(node, null, null, null)

    // expand and turn into a wide data row
    // have to use 2x map b/c reduce was giving me weird errors
    const keys = quads.map(r => r._predicate.id.split("#")[1])
    const values = quads.map(r => r._object.id)

    return { id: node.id, ...createObject(keys, values) }
}

function getTopLevelResults(store){
    /// Returns all top level validation results
    return store.getObjects(null, namedNode("http://www.w3.org/ns/shacl#result"), null)
}

function getResultDetails(store, node){
    /// for the given node, get all validation result details, if present
    return store.getObjects(node, namedNode("http://www.w3.org/ns/shacl#detail"), null)
}


// Util
function createObject(keys, values) {
    // Given list of keys and values, zip into object.
    const obj = Object.fromEntries(
        keys.map((key, index) => [key, values[index]]),
    );

    return obj;
}


In [28]:
function generateValidationResultTreeData(store, rootAtFocus=true){
    /// Given a graph (store) of a valid SHACL result graph; generate an ag-grid compatible tree ;
    /// Additionally enrich nodes with data for grid columns.
    let results = []

    for (let node of getTopLevelResults(store)) {
        let path = []
        // seed path
        if(rootAtFocus) { path.push( store.getObjects(node, namedNode("http://www.w3.org/ns/shacl#focusNode"), null)?.[0]?.id ) }
        path.push(node.id)

        results.push({path, ...enrichNode(store, node)})

        // recurse the details
        _recursivelyGetDetail(store, node, path, results)
    }

    return results
}

function _recursivelyGetDetail(store, node, path, results){
    // let result = []
    // given a node, get detail if exists
    const detailNodes = getResultDetails(store, node)

    for (let detailNode of detailNodes){
        const newPath = [...path, detailNode.id]
        results.push({path: newPath, ...enrichNode(store, detailNode)})

        _recursivelyGetDetail(store, detailNode, newPath, results)
    }
}


In [10]:
let topLevelNodes = getTopLevelResults(graphStore.graph)

In [11]:
enrichNode(graphStore.graph, topLevelNodes[0])

{
  id: "_:n3-1",
  type: "http://www.w3.org/ns/shacl#ValidationResult",
  focusNode: "uri:ex#DB3A04F0-5109-4514-9192-434AA2D34ADE",
  resultMessage: '"Value does not conform to Shape plV:Zone_Node_Cardinality. See details for more information."',
  resultSeverity: "http://www.w3.org/ns/shacl#Violation",
  sourceConstraintComponent: "http://www.w3.org/ns/shacl#NodeConstraintComponent",
  sourceShape: "https://www.passivelogic.com/schema/EntityValidationShape#Zone_Atomics",
  detail: "_:n3-2",
  value: "uri:ex#DB3A04F0-5109-4514-9192-434AA2D34ADE"
}

In [12]:
getResultDetails(graphStore.graph, topLevelNodes[0]).forEach((res)=>{ console.log(enrichNode(graphStore.graph, res))})

{
  id: "_:n3-2",
  type: "http://www.w3.org/ns/shacl#ValidationResult",
  focusNode: "uri:ex#DB3A04F0-5109-4514-9192-434AA2D34ADE",
  resultMessage: '"Zone does not have a volume property"',
  resultPath: "https://www.passivelogic.com/schema/core#hasProperty",
  resultSeverity: "http://www.w3.org/ns/shacl#Violation",
  sourceConstraintComponent: "http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent",
  sourceShape: "_:b0_n8b1237bed3344264a4681a4ea0713f1eb14"
}


In [13]:
topLevelNodes[0].id

"_:n3-1"

In [29]:
const res = generateValidationResultTreeData(graphStore.graph)

In [30]:
await Deno.writeTextFile("test_data.json", JSON.stringify(res))